<a href="https://colab.research.google.com/github/danielpatrickhug/Fitting_some_sets/blob/main/arxiv_graph_laplacian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%%bash
pip install sentence-transformers

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import scipy
import scipy.stats
import json

model = SentenceTransformer('allenai-specter')
arxiv_path = "/content/drive/MyDrive/arxiv_distance_graph/data/arxiv-metadata-oai-snapshot.json"

def degree_matrix(A):
    """
    compute degree matrix using adjacency distance matrix from pairwise distances
    :A: nxn size matrix embedding minmaxed using mu sigma and pairwise distances
    :return: degree matrix
    """
    n = A.shape[0]
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = np.sum(A[i, :])
    return D

def graph_laplacian(A):
    """
    compute graph laplacian using degree and adjacency matrix from pairwise distances
    :A: nxn size matrix embedding minmaxed using mu sigma and pairwise distances
    :return: graph laplacian, and degree matrix
    """
    D = degree_matrix(A)
    L = D - A
    return L, D

def cosine_similarity(a, b):
    """
    inner product between two vectors divided by the norm
    :a: n-dim feature vector
    :b: n-dim feature vector
    :return: the cosine similarity of the two embeddings
    """
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def get_article_embeddings_matrix(train_docs_list):
    """
    extract features and merge feature matrix for given documents
    :train_doc_list: list of training documents
    :return: nd.array feature matrix of size (embed dim, number of articles)
    """
    embeddings = model.encode(train_docs_list, convert_to_numpy=True, device='cuda')
    return np.array(embeddings).T


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

In [ ]:
def get_all_topics(data_path: str):
    """
    Load all research topic query strings 
    :data_path: path to arxiv file
    :return: return unique list of topic query strings
    """
    def flatten(t: list) -> list:
        return [item for sublist in t for item in sublist]

    topics = []
    with open(data_path, "r") as f:
        for l in f:
            d = json.loads(l)
            topics.append(d['categories'].split(' '))
                    
    topics = flatten(topics)
    return list(set(topics))


def get_topic_data(data_path: str, topic: str):
    """
    Load arxiv articles from the given topic
    :data_path: path to axiv_json file
    :topic: topic string of research topics
    :return: return articles for a given topic
    """
    articles = []
    with open(data_path, "r") as f:
        for l in f:
            d = json.loads(l)
            if topic in d['categories'].split(' ')[0]:
                    articles.append(d['abstract'])

    return articles

In [ ]:
topics = get_all_topics(arxiv_path)

In [ ]:
math_topics = [topic for topic in topics if 'math' in topic]
math_topics

['math.GT',
 'math.GM',
 'math.RT',
 'math.CA',
 'math.NT',
 'math.ST',
 'math.QA',
 'math.PR',
 'math.AT',
 'math.IT',
 'math.KT',
 'math.AC',
 'math.MG',
 'math.MP',
 'math.FA',
 'math.NA',
 'math.AP',
 'math.DG',
 'math.GR',
 'math.LO',
 'math.CV',
 'math.CT',
 'math-ph',
 'math.DS',
 'math.CO',
 'math.GN',
 'math.SG',
 'math.HO',
 'math.SP',
 'math.OA',
 'math.RA',
 'math.OC',
 'math.AG']

In [ ]:
def estimate_pdf(scores: list):
    """
    estimate scores probability density function
    :scores: list of distance scores from topic features to topic centroid
    :return: distribution
    """
    return scipy.stats.gaussian_kde(scores)

def calc_feature_centroid_distances(topic_embeddings, centroid):
    """
    compute distances for each topic feature and the topic centroid
    :topic_embeddings: sentence embeddings matrix for a topic
    :centroid: topic centroid
    :return: list of distance scores
    """
    return [cosine_similarity(doc, centroid).item() for doc in topic_embeddings.T.tolist()]

def calc_sigma(scores: list):
    """
    calc standard deviation of scores
    :scores: list of distances for the topic features and their centroid
    :return: standard deviation of scores list
    """
    return np.std(scores)

def learn_centroid(topic_embeddings):
    """
    learn centroid from topic embeddings by summing vectors
    :topic_embeddings: sentence embeddings matrix for a topic
    :return: topic centroid
    """
    return np.sum(topic_embeddings, axis=1, keepdims=True)



def get_topic_embeddings(topic: str, topics_size):
    """
    for topic compute the embeddings matrix for a given topic size
    :topic: arxiv topic string
    :topics_size: number of articles to embed from topic abstracts
    :return: embeddings, centroid, mode, and sigma of the topic embeddings
    """
    train_data = get_topic_data(arxiv_path, topic)
    train_topic_embeddings = get_article_embeddings_matrix(train_data[:topics_size])

    centroid = learn_centroid(train_topic_embeddings)
    scores = calc_feature_centroid_distances(train_topic_embeddings, centroid)
    distribution = estimate_pdf(scores)
    mode = calc_shgo_mode(scores,distribution)
    sigma = calc_sigma(scores)
    return train_topic_embeddings, centroid, mode, sigma


def feature_matrix(topics, topics_size):
    """
    for the given arxiv topics compute full feature matrix, topic centroids, topic mode/sigmas, and the index for B
    :topics: topic query strings for getting articles from json file
    :topics_size: 
    """
    topic_modes = {}
    topic_sigmas = {}
    centroids = {}
    embeddings = []
    feature_matrix_index = {}
    curr_index = 0

    for topic in topics:
        print(topic)
        topic_embeddings, centroid, mode, sigma = get_topic_embeddings(topic, topics_size)
        print(topic_embeddings.shape)
        embeddings.append(topic_embeddings.T)
        centroids[topic] = centroid
        topic_modes[topic] = mode
        topic_sigmas[topic] = sigma
        #
        for i in range(curr_index, topics_size+curr_index):
            print(i, topic)
            feature_matrix_index[i] = topic
        curr_index += topics_size

    B = np.concatenate(embeddings)
    return B, centroids, topic_modes, topic_sigmas, feature_matrix_index

In [ ]:
sentence_embeddings, centroids, topic_modes, topic_sigma, feature_matrix_index = feature_matrix(math_topics[:5], topics_size=30)

math.GT
(768, 30)
0 math.GT
1 math.GT
2 math.GT
3 math.GT
4 math.GT
5 math.GT
6 math.GT
7 math.GT
8 math.GT
9 math.GT
10 math.GT
11 math.GT
12 math.GT
13 math.GT
14 math.GT
15 math.GT
16 math.GT
17 math.GT
18 math.GT
19 math.GT
20 math.GT
21 math.GT
22 math.GT
23 math.GT
24 math.GT
25 math.GT
26 math.GT
27 math.GT
28 math.GT
29 math.GT
math.GM
(768, 30)
30 math.GM
31 math.GM
32 math.GM
33 math.GM
34 math.GM
35 math.GM
36 math.GM
37 math.GM
38 math.GM
39 math.GM
40 math.GM
41 math.GM
42 math.GM
43 math.GM
44 math.GM
45 math.GM
46 math.GM
47 math.GM
48 math.GM
49 math.GM
50 math.GM
51 math.GM
52 math.GM
53 math.GM
54 math.GM
55 math.GM
56 math.GM
57 math.GM
58 math.GM
59 math.GM


In [ ]:
sentence_embeddings.shape

(60, 768)

In [ ]:
feature_matrix_index

{0: 'math.GT',
 1: 'math.GT',
 2: 'math.GT',
 3: 'math.GT',
 4: 'math.GT',
 5: 'math.GT',
 6: 'math.GT',
 7: 'math.GT',
 8: 'math.GT',
 9: 'math.GT',
 10: 'math.GT',
 11: 'math.GT',
 12: 'math.GT',
 13: 'math.GT',
 14: 'math.GT',
 15: 'math.GT',
 16: 'math.GT',
 17: 'math.GT',
 18: 'math.GT',
 19: 'math.GT',
 20: 'math.GT',
 21: 'math.GT',
 22: 'math.GT',
 23: 'math.GT',
 24: 'math.GT',
 25: 'math.GT',
 26: 'math.GT',
 27: 'math.GT',
 28: 'math.GT',
 29: 'math.GT',
 30: 'math.GM',
 31: 'math.GM',
 32: 'math.GM',
 33: 'math.GM',
 34: 'math.GM',
 35: 'math.GM',
 36: 'math.GM',
 37: 'math.GM',
 38: 'math.GM',
 39: 'math.GM',
 40: 'math.GM',
 41: 'math.GM',
 42: 'math.GM',
 43: 'math.GM',
 44: 'math.GM',
 45: 'math.GM',
 46: 'math.GM',
 47: 'math.GM',
 48: 'math.GM',
 49: 'math.GM',
 50: 'math.GM',
 51: 'math.GM',
 52: 'math.GM',
 53: 'math.GM',
 54: 'math.GM',
 55: 'math.GM',
 56: 'math.GM',
 57: 'math.GM',
 58: 'math.GM',
 59: 'math.GM'}

In [ ]:
topic_modes

{'math.GM': 0.8642499477447796, 'math.GT': 0.9017918738447788}

In [ ]:
topic_sigma

{'math.GM': 0.046210518413574965, 'math.GT': 0.032606930120146774}

In [ ]:
def degree_matrix(A):
    """
    compute degree matrix using adjacency distance matrix from pairwise distances
    :A: nxn size matrix embedding minmaxed using mu sigma and pairwise distances
    :return: degree matrix
    """
    n = A.shape[0]
    D = np.zeros((n, n))
    for i in range(n):
        D[i, i] = np.sum(A[i, :])
    return D

def graph_laplacian(A):
    """
    compute graph laplacian using degree and adjacency matrix from pairwise distances
    :A: nxn size matrix embedding minmaxed using mu sigma and pairwise distances
    :return: graph laplacian, and degree matrix
    """
    D = degree_matrix(A)
    L = D - A
    return L, D

def cosine_similarity(a, b):
    """
    inner product between two vectors divided by the norm
    :a: n-dim feature vector
    :b: n-dim feature vector
    :return: the cosine similarity of the two embeddings
    """
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def adjaceny_matrix(sentence_embeddings, topic_index, topic_mu, topic_sigma):
    A = np.zeros((sentence_embeddings.shape[0], sentence_embeddings.shape[0]))
    for i in range(sentence_embeddings.shape[0]):
        i_topic = topic_index[i]
        for j in range(sentence_embeddings.shape[0]):
            i_j_sim = cosine_similarity(sentence_embeddings[i], sentence_embeddings[j])
            print(i_j_sim)
            j_topic = topic_index[j]
            print(j_topic)
            if i == j:
                A[i, j] = 0
            elif i_j_sim > topic_mu[i_topic] - 3*topic_sigma[i_topic]:
                A[i, j] = 1
            else:
                A[i, j] = 0
    return A

In [ ]:
A = adjaceny_matrix_kernel_v0(sentence_embeddings, feature_matrix_index, topic_modes, topic_sigma)
A

Streaming output truncated to the last 5000 lines.
0.67217666
math.GT
0.6963351
math.GT
match
0.7010187
math.GT
match
0.65670264
math.GT
0.76131684
math.GT
match
0.7090148
math.GT
match
0.7439698
math.GT
match
0.7290045
math.GT
match
0.6800588
math.GT
match
0.6903335
math.GT
match
0.7114749
math.GT
match
0.6248089
math.GT
0.76085085
math.GT
match
0.71361053
math.GT
match
0.7209522
math.GT
match
0.7125488
math.GT
match
0.64625984
math.GT
0.7116955
math.GT
match
0.7432512
math.GT
match
0.760275
math.GT
match
0.69718206
math.GT
match
0.67375207
math.GT
1.0
math.GM
here
0.75050676
math.GM
match
0.66177475
math.GM
0.7899988
math.GM
match
0.7470923
math.GM
match
0.71216
math.GM
match
0.7535892
math.GM
match
0.6254325
math.GM
0.73434424
math.GM
match
0.71394175
math.GM
match
0.66362894
math.GM
0.76327884
math.GM
match
0.8010432
math.GM
match
0.750551
math.GM
match
0.71229315
math.GM
match
0.72253215
math.GM
match
0.7645127
math.GM
match
0.7264765
math.GM
match
0.803246
math.GM
match
0.6334221

array([[0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [ ]:
print(f'A: {A}\n')
#compute graph laplacian and degree matrix
L, D = graph_laplacian(A)

print(f'D: {D}\n')
print(f'L: {L}\n')
U, S, VT = np.linalg.svd(L)
print(f'U: {U}\n')
print(f'S: {S}\n')
print(f'VT: {VT}\n')

A: [[0. 1. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 1. ... 0. 0. 1.]
 [1. 1. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]]

D: [[23.  0.  0. ...  0.  0.  0.]
 [ 0. 25.  0. ...  0.  0.  0.]
 [ 0.  0.  7. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 30.  0.  0.]
 [ 0.  0.  0. ...  0. 50.  0.]
 [ 0.  0.  0. ...  0.  0. 58.]]

L: [[23. -1.  0. ...  0.  0. -1.]
 [-1. 25.  0. ...  0. -1.  0.]
 [ 0.  0.  7. ...  0.  0. -1.]
 ...
 [ 0.  0. -1. ... 30.  0. -1.]
 [-1. -1.  0. ...  0. 50. -1.]
 [-1. -1. -1. ... -1. -1. 58.]]

U: [[-1.64546432e-02  2.31994239e-03  6.32744161e-03 ...  4.21017453e-02
  -5.98673544e-03 -1.67855227e-01]
 [ 5.74572023e-04  7.26733427e-04 -7.64991152e-03 ...  4.62120691e-02
  -6.38741596e-03 -1.72322293e-01]
 [-9.00459057e-03 -1.29919039e-02  1.62287535e-02 ...  1.21753892e-01
  -6.15837553e-04 -2.39287598e-01]
 ...
 [-6.31680675e-04 -3.54485998e-03  2.38493390e-02 ...  7.64894618e-03
  -4.66312249e-04 -2.64427362e-02]
 [ 1.65015100e-03 -6

experiment

In [ ]:
#get graph embedding
graph_embedding0 = np.dot(U, np.dot(np.diag(S), VT))
print(f'GE_0: {graph_embedding0.shape}')

graph_embedding1, avg_doc, U_t = sum([S[i]*np.outer(U[:,i], VT[i,:]) for i in range(c)])
print(f'GE_1: {graph_embedding1.shape}')

distances = []
for i in range(graph_embedding0.shape[0]):
    for j in range(graph_embedding0.shape[0]):
        if i==j:
            print(cosine_similarity(graph_embedding0[i], graph_embedding0[j]))

GE_0: (60, 60)
GE_1: (40, 60)
0.9999999999999998
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0
1.0
0.9999999999999998
1.0
0.9999999999999999
0.9999999999999999
1.0
1.0000000000000002
1.0
1.0000000000000002
1.0
1.0
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
1.0
1.0
1.0000000000000002
0.9999999999999999
1.0
1.0
1.0
1.0000000000000002
1.0
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999998
1.0
1.0000000000000002
0.9999999999999998
0.9999999999999999
1.0000000000000002
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0000000000000002
1.0
0.9999999999999999
